In [30]:
# Data manipulation
from seaborn import load_dataset
import numpy as np
import pandas as pd
pd.options.display.precision = 4
pd.options.mode.chained_assignment = None  

# Machine learning pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn import set_config
set_config(display="diagram")

# Load data
columns = ['alive', 'class', 'embarked', 'who', 'alone', 'adult_male']
df = load_dataset('titanic').drop(columns=columns)
df['deck'] = df['deck'].astype('object')
print(df.shape)
df.head()

(891, 9)


,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton
3,1,1,female,35.0,1,0,53.1000,C,Southampton
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton


In [31]:
SEED = 42
TARGET = 'survived'
FEATURES = df.columns.drop(TARGET)

NUMERICAL = df[FEATURES].select_dtypes('number').columns
print(f"Numerical features: {', '.join(NUMERICAL)}")

CATEGORICAL = pd.Index(np.setdiff1d(FEATURES, NUMERICAL))
print(f"Categorical features: {', '.join(CATEGORICAL)}")

Numerical features: pclass, age, sibsp, parch, fare
Categorical features: deck, embark_town, sex


### 1. Неправильный подход

In [32]:
# Impute numerical variables with mean
df_num_imputed = df[NUMERICAL].fillna(df[NUMERICAL].mean())
# Normalise numerical variables
df_num_scaled = df_num_imputed.subtract(df_num_imputed.min(), axis=1)\
                              .divide(df_num_imputed.max()-df_num_imputed.min(), axis=1)

# Impute categorical variables with a constant
df_cat_imputed = df[CATEGORICAL].fillna('missing')
# One-hot-encode categorical variables
df_cat_encoded = pd.get_dummies(df_cat_imputed, drop_first=True)

# Merge data
df_preprocessed = df_num_scaled.join(df_cat_encoded)
df_preprocessed.head()

,pclass,age,sibsp,parch,fare,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_missing,embark_town_Queenstown,embark_town_Southampton,embark_town_missing,sex_male
0,1.0,0.2712,0.125,0.0,0.0142,False,False,False,False,False,False,True,False,True,False,True
1,0.0,0.4722,0.125,0.0,0.1391,False,True,False,False,False,False,False,False,False,False,False
2,1.0,0.3214,0.000,0.0,0.0155,False,False,False,False,False,False,True,False,True,False,False
3,0.0,0.4345,0.125,0.0,0.1036,False,True,False,False,False,False,False,False,True,False,False
4,1.0,0.4345,0.000,0.0,0.0157,False,False,False,False,False,False,True,False,True,False,True


Мы вменили недостающие значения, масштабировали числовые переменные от 0 до 1 и закодировали категориальные переменные методом горячего кодирования. После предварительной обработки данные секционируются и подгоняется модель:

In [33]:
# Partition data
X_train, X_test, y_train, y_test = train_test_split(df_preprocessed, df[TARGET], 
                                                    test_size=.2, random_state=SEED, 
                                                    stratify=df[TARGET])

# Train a model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

Хорошо, давайте проанализируем, что не так с этим подходом:
   
◼️ Вменение: числовые переменные следует вменять со средним значением обучающих данных, а не со всеми данными.   
◼️ Масштабирование: минимальное и максимальное значения следует рассчитывать на основе данных трениров.    
   .
◼️ Кодирование: категории следует определять на основе обучающих данных. Кроме того, даже если данные секционированы до предварительной обработки, горячее кодирование с помощью pd.get_dummies(X_train)и pd.get_dummies(X_test)может привести к противоречивым данным обучения и тестирования (т. е. столбцы могут различаться в зависимости от категорий в обоих наборах данных). Поэтому pd.get_dummies()не следует использовать горячее кодирование при подготовке данных для модели   .

💡 Данные испытаний следует сохранить до предварительной обработки. Любая статистика, такая как среднее, минимальное и максимальное значение, используемая для предварительной обработки, должна быть получена из обучающих данных. В противном случае возникнет проблема утечки данных.

Теперь давайте оценим модель. Мы будем использовать ROC-AUC для оценки модели. Мы создадим функцию, которая вычисляет ROC-AUC, поскольку она будет полезна для оценки последующих подходов:

In [34]:
def calculate_roc_auc(model_pipe, X, y):
    """Calculate roc auc score. 
    
    Parameters:
    ===========
    model_pipe: sklearn model or pipeline
    X: features
    y: true target
    """
    y_proba = model_pipe.predict_proba(X)[:,1]
    return roc_auc_score(y, y_proba)
  
print(f"Train ROC-AUC: {calculate_roc_auc(model, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(model, X_test, y_test):.4f}")

Train ROC-AUC: 0.8669
Test ROC-AUC: 0.8329


❔2. Правильный подход, но…
Сначала мы разделим данные и предварительно обработаем их с помощью преобразователей Scikit-learn, чтобы предотвратить утечку данных за счет правильной предварительной обработки:

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=TARGET), df[TARGET], 
                                                    test_size=.2, random_state=SEED, 
                                                    stratify=df[TARGET])
num_imputer = SimpleImputer(strategy='mean')
train_num_imputed = num_imputer.fit_transform(X_train[NUMERICAL])

scaler = MinMaxScaler()
train_num_scaled = scaler.fit_transform(train_num_imputed)

cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
train_cat_imputed = cat_imputer.fit_transform(X_train[CATEGORICAL])

encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
train_cat_encoded = encoder.fit_transform(train_cat_imputed)

train_preprocessed = np.concatenate((train_num_scaled, train_cat_encoded), axis=1)

columns = np.append(NUMERICAL, encoder.get_feature_names_out(CATEGORICAL))
pd.DataFrame(train_preprocessed, columns=columns, index=X_train.index).head()

/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,pclass,age,sibsp,parch,fare,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_missing,embark_town_Queenstown,embark_town_Southampton,embark_town_missing,sex_male
692,1.0,0.3693,0.000,0.0000,0.1103,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
481,0.5,0.3693,0.000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
527,0.0,0.3693,0.000,0.0000,0.4329,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
855,1.0,0.2209,0.000,0.1667,0.0182,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
801,0.5,0.3843,0.125,0.1667,0.0512,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [45]:
train_cat_encoded

array([[0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.]])

Прекрасно, теперь мы можем подогнать модель:

In [16]:
model = LogisticRegression()
model.fit(train_preprocessed, y_train)

LogisticRegression()

Перед оценкой нам необходимо таким же образом предварительно обработать тестовый набор данных:

In [17]:
test_num_imputed = num_imputer.transform(X_test[NUMERICAL])
test_num_scaled = scaler.transform(test_num_imputed)
test_cat_imputed = cat_imputer.transform(X_test[CATEGORICAL])
test_cat_encoded = encoder.transform(test_cat_imputed)
test_preprocessed = np.concatenate((test_num_scaled, test_cat_encoded), axis=1)

print(f"Train ROC-AUC: {calculate_roc_auc(model, train_preprocessed, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(model, test_preprocessed, y_test):.4f}")

Train ROC-AUC: 0.8670
Test ROC-AUC: 0.8332


✔️3. Элегантный подход №1
Давайте упростим предыдущий код, используя Scikit-learn Pipelineи ColumnTransformer. Если вы с ними не знакомы, в этом посте они кратко описаны.

In [18]:
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False))
])

preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, NUMERICAL),
    ('cat', categorical_pipe, CATEGORICAL)
])

pipe = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LogisticRegression())
])

pipe.fit(X_train, y_train)

/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessors',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['deck', 'embark_town', 'sex'], dtype='object'))])),
                ('model', LogisticRegression())])

Конвейер:
◼️ Разбивает входные данные на числовые и категориальные группы
◼️ Предварительно обрабатывает обе группы параллельно
◼️ Объединяет предварительно обработанные данные из обеих групп
◼️ Передает предварительно обработанные данные в модель

Когда необработанные данные передаются в обученный конвейер, он предварительно обрабатывает и делает прогноз. Это означает, что нам больше не нужно хранить промежуточные результаты как для обучающего, так и для тестового набора данных. Оценить невидимые данные так же просто, как pipe.predict(). Это очень элегантно, не так ли? Теперь оценим производительность модели:

In [19]:
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8670
Test ROC-AUC: 0.8332


✔️ 4. Элегантный подход №2
При таком подходе мы создадим собственные преобразователи с помощью Scikit-learn. Мы надеемся, что наблюдение за тем, как те же этапы предварительной обработки, с которыми мы ознакомились, переводятся в пользовательские преобразователи, поможет вам понять основную идею их создания. Если вас интересуют примеры вариантов использования пользовательских преобразователей, посетите этот репозиторий GitHub .

In [20]:
class Imputer(BaseEstimator, TransformerMixin):
    def __init__(self, features, method='constant', value='missing'):
        self.features = features
        self.method = method
        self.value = value
    
    def fit(self, X, y=None):
        if self.method=='mean':
            self.value = X[self.features].mean()
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.features] = X[self.features].fillna(self.value)
        return X_transformed
    
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        self.min = X[self.features].min()
        self.range = X[self.features].max()-self.min
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.features] = (X[self.features]-self.min)/self.range
        return X_transformed
  
class Encoder(BaseEstimator, TransformerMixin):
    def __init__(self, features, drop='first'):
        self.features = features
        self.drop = drop
    
    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse=False, drop=self.drop)
        self.encoder.fit(X[self.features])
        return self
    
    def transform(self, X):
        X_transformed = pd.concat([X.drop(columns=self.features).reset_index(drop=True), 
                                   pd.DataFrame(self.encoder.transform(X[self.features]), 
                                                columns=self.encoder.get_feature_names_out(self.features))],
                                  axis=1)
        return X_transformed
        
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('model', LogisticRegression())
])

pipe.fit(X_train, y_train)   

/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'),
                         method='mean',
                         value=pclass     2.3090
age       29.8077
sibsp      0.4930
parch      0.3904
fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('model', LogisticRegression())])

В отличие от предыдущего, шаги выполняются последовательно один за другим, каждый шаг передает выходные данные следующему шагу в качестве входных данных. Пришло время оценить модель:

In [21]:

print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8670
Test ROC-AUC: 0.8332


Для добавления класса, который будет подбирать параметры логистической регрессии, вы можете создать новый класс, например, LogisticRegressionTuner, который будет выполнять поиск оптимальных параметров для модели. Затем этот класс может быть включен в вашу конвейерную обработку.

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

class LogisticRegressionTuner(BaseEstimator, TransformerMixin):
    def __init__(self, param_grid=None, cv=3):
        self.param_grid = param_grid
        self.cv = cv
    
    def fit(self, X, y=None):
        self.model = LogisticRegression()
        self.grid_search = GridSearchCV(self.model, self.param_grid, cv=self.cv)
        self.grid_search.fit(X, y)
        return self
    
    def transform(self, X):
        # LogisticRegressionTuner не изменяет данные, поэтому просто возвращаем X
        return X

# Определите сетку параметров, которые вы хотите оптимизировать для LogisticRegression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2']
}

# Добавьте LogisticRegressionTuner в ваш конвейер
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('logistic_regression_tuner', LogisticRegressionTuner(param_grid=param_grid, cv=3)),
    ('model', LogisticRegression())  # Добавьте оптимизированную модель
])

# Запустите конвейер на тренировочных данных
pipe.fit(X_train, y_train)


/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'),
                         method='mean',
                         value=pclass     2.3090
age       29.8077
sibsp      0.4930
parch      0.3904
fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('logistic_regression_tuner',
                 LogisticRegressionTuner(param_grid={'C': [0.001, 0.01, 0.1, 1,
                                                           10, 100],
                                                     'penalty': ['l2']})),
                ('model', LogisticRegression())])

In [23]:
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8670
Test ROC-AUC: 0.8332


Для тренировки нейронной сети с использованием TensorFlow, вы можете создать соответствующий класс, который будет выполнять обучение и применение вашей нейронной сети. Затем этот класс может быть включен в ваш конвейер. Вот пример конвейера для обучения нейронной сети с использованием TensorFlow:

In [24]:
import tensorflow as tf
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class NeuralNetworkTrainer(BaseEstimator, TransformerMixin):
    def __init__(self, epochs=10, batch_size=32, verbose=1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
    
    def fit(self, X, y=None):
        input_shape = (X.shape[1],)  # Определите форму входных данных на основе X
        num_classes = len(np.unique(y))  # Количество классов в задаче классификации
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(2048, activation='relu', input_shape=input_shape),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self.model = model
        return self
    
    def transform(self, X):
        # NeuralNetworkTrainer не изменяет данные, поэтому просто возвращаем X
        return X

    def predict(self, X):
        # Предсказать метки классов для новых данных X
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
        
    def score(self, X, y=None):
        # Оцените производительность модели на данных X и y
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        accuracy = accuracy_score(y, np.argmax(y_pred, axis=1))
        return accuracy

# Добавьте NeuralNetworkTrainer в ваш конвейер
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('neural_network_trainer', NeuralNetworkTrainer()),
])

# Запустите конвейер на тренировочных данных
pipe.fit(X_train, y_train)




2023-09-11 21:57:54.483716: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:57:54.703782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:57:54.705549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 21:57:56.040865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
23/23 [==============================] - 1s 6ms/step - loss: 0.5468 - accuracy: 0.7121
Epoch 2/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4656 - accuracy: 0.7893
Epoch 3/10
23/23 [==============================] - 0s 5ms/step - loss: 0.4486 - accuracy: 0.7879
Epoch 4/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4399 - accuracy: 0.7992
Epoch 5/10
23/23 [==============================] - 0s 5ms/step - loss: 0.4294 - accuracy: 0.8202
Epoch 6/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4241 - accuracy: 0.8062
Epoch 7/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4192 - accuracy: 0.8244
Epoch 8/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4089 - accuracy: 0.8315
Epoch 9/10
23/23 [==============================] - 0s 5ms/step - loss: 0.4030 - accuracy: 0.8287
Epoch 10/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4054 - accuracy: 0.8272


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'),
                         method='mean',
                         value=pclass     2.3090
age       29.8077
sibsp      0.4930
parch      0.3904
fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['deck', 'embark_town', 'sex'], dtype='object'))),
                ('neural_network_trainer', NeuralNetworkTrainer())])

In [25]:

# Сделайте предсказание на тестовых данных
y_pred = pipe.predict(X_test)


6/6 [==============================] - 0s 4ms/step


In [26]:
from sklearn.metrics import accuracy_score

# Вычислите оценку точности (accuracy) модели
accuracy = accuracy_score(y_test, y_pred)

# Выведите оценку на экран
print("Accuracy:", accuracy)


Accuracy: 0.7877094972067039


In [27]:
import tensorflow as tf
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

class NeuralNetworkTrainer(BaseEstimator, TransformerMixin):
    def __init__(self, epochs=10, batch_size=32, verbose=1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
    
    def fit(self, X, y=None):
        input_shape = (X.shape[1],)  # Определите форму входных данных на основе X
        num_classes = len(np.unique(y))  # Количество классов в задаче классификации
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(2048, activation='relu', input_shape=input_shape),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self.model = model
        return self
    
    def transform(self, X):
        # NeuralNetworkTrainer не изменяет данные, поэтому просто возвращаем X
        return X

    def predict(self, X):
        # Предсказать метки классов для новых данных X
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
        
    def score(self, X, y=None):
        # Оцените производительность модели на данных X и y
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        accuracy = accuracy_score(y, np.argmax(y_pred, axis=1))
        return accuracy

# Остальной код остается без изменений

# Создайте экземпляр нейронной сети
neural_network = NeuralNetworkTrainer()

# Определите сетку параметров для нейронной сети
param_grid = {
    'neural_network__epochs': [10, 20, 30],  # Разные значения числа эпох
    'neural_network__batch_size': [32, 64, 128],  # Разные значения размера пакета
    'neural_network__verbose': [0, 1],  # Разные уровни вывода
}

# Создайте конвейер с нейронной сетью
pipeline = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('neural_network', neural_network),
])

# Создайте GridSearchCV объект с конвейером и сеткой параметров
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1)

# Запустите поиск по сетке на тренировочных данных
grid_search.fit(X_train, y_train)

# Получите наилучшие параметры
best_params = grid_search.best_params_
print("Наилучшие параметры:", best_params)


2023-09-11 21:58:12.067288: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:58:12.076313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:58:12.142327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:58:12.143524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 21:58:12.155534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 21:58:12.156272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instru

Epoch 1/10


/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed 

Epoch 1/10


/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/gea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


...

4/4 [==============================] - 0s 8ms/step - loss: 0.3885 - accuracy: 0.8421
Epoch 27/30
Epoch 24/30
4/4 [==============================] - 0s 12ms/step - loss: 0.3965 - accuracy: 0.8421
Epoch 27/30
4/4 [==============================] - 0s 8ms/step - loss: 0.3861 - accuracy: 0.8337
Epoch 25/30
4/4 [==============================] - 0s 10ms/step - loss: 0.3826 - accuracy: 0.8397
Epoch 28/30
4/4 [==============================] - 0s 9ms/step - loss: 0.3941 - accuracy: 0.8463
Epoch 28/30
4/4 [==============================] - 0s 10ms/step - loss: 0.3832 - accuracy: 0.8400
Epoch 26/30
4/4 [==============================] - 0s 8ms/step - loss: 0.3806 - accuracy: 0.8439
Epoch 29/30
4/4 [==============================] - 0s 10ms/step - loss: 0.3923 - accuracy: 0.8442
Epoch 29/30
4/4 [==============================] - 0s 10ms/step - loss: 0.3822 - accuracy: 0.8421
Epoch 27/30
4/4 [==============================] - 0s 13ms/step - loss: 0.3795 - accuracy: 0.8502
Epoch 30/30
4/4 [=======

/home/gea/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/pipeline.py", line 749, in score
    Xt = transform.transform(Xt)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/tmp/ipykernel_9164/2555258950.py", line 43, in transform
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/utils/_set_output.py", line 140, in wrappe

4/4 [==============================] - 0s 9ms/step - loss: 0.3777 - accuracy: 0.8442
Epoch 29/30
4/4 [==============================] - 0s 7ms/step - loss: 0.3737 - accuracy: 0.8505
Epoch 30/30
8/8 [==============================] - 0s 2ms/step


/home/gea/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/pipeline.py", line 749, in score
    Xt = transform.transform(Xt)
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/tmp/ipykernel_9164/2555258950.py", line 43, in transform
  File "/home/gea/.local/lib/python3.8/site-packages/sklearn/utils/_set_output.py", line 140, in wrappe

Наилучшие параметры: {'neural_network__batch_size': 32, 'neural_network__epochs': 10, 'neural_network__verbose': 0}


In [28]:
# Сделайте предсказание на тестовых данных
y_pred = pipe.predict(X_test)

6/6 [==============================] - 0s 2ms/step


In [29]:
from sklearn.metrics import accuracy_score

# Вычислите оценку точности (accuracy) модели
accuracy = accuracy_score(y_test, y_pred)

# Выведите оценку на экран
print("Accuracy:", accuracy)

Accuracy: 0.7877094972067039
